# Chapter 2. 파이썬스러운(pythonic) 코드
+ 이 장의 목표
    + 인덱스와 슬라이스를 이해하고 인덱싱 가능한 객체를 올바른 방식으로 구현
    + 시퀀스와 이터러블 구현
    + 매직 메서드를 사요해 보다 관용적인 코드 구현
    + 부작용을 유발하는 흔한 실수 피하기

## 인덱스와 슬라이스

In [1]:
my_numbers = (4, 5, 3, 9)

print("음수 인덱스:", my_numbers[-1], my_numbers[-3])
print("slice 1:", my_numbers[2:5])
print("slice 2:", my_numbers[:3])
print("slice 3:", my_numbers[3:])
print("slice 4:", my_numbers[::-1])

print("\n")
print("slice index:", my_numbers[1:7:2])
print("slice object:", my_numbers[slice(1, 7, 2)])

print("\n")
interval = slice(None, 3)
print("slice object vs index:", my_numbers[interval] == my_numbers[:3])

음수 인덱스: 9 5
slice 1: (3, 9)
slice 2: (4, 5, 3)
slice 3: (9,)
slice 4: (9, 3, 5, 4)


slice index: (5, 9)
slice object: (5, 9)


slice object vs index: True


## 자체 시퀀스 생성
+ 범위로 인덱싱하는 결과는 해당 클래스와 같은 타입의 인스턴스여아 한다.
+ slice에 의해 제공된 범위는 파이썬이 하는 것처럼 마지막 요소는 제외한다.

In [2]:
""" list wrapping 사례 """

class Item:
    def __init__(self, *values):
        self._value = list(values)
    
    def __len__(self):
        return len(self._value)

    def __getitem__(self, index):
        return self._value.__getitem__(index)

In [3]:
item = Item(3, 4, 5)
print(len(item))
print(item[1:])
print(type(item[1:]))

3
[4, 5]
<class 'list'>


## 컨텍스트 관리자
+ __enter__와 __exit__ 두 개의 매직 메서드로 구성됨
+ with 문은 __enter__ 메서드를 호추랗고 이 메서드가 무엇을 반환하든 as 이후에 지정된 변수에 할당됨
+ 해당 블록에 대한 마지막 문장이 끝나면 컨텍스트가 종료되며 __exit__ 메서드를 호출함
+ 블록 내에 예외 또는 오류가 있는 경우에도 __exit__ 메서드가 여전히 호출되며, 예외는 파라미터로 확인 가능
    + True를 반환하면 잠재적으로 발생한 예외를 호출자에게 전파하지 않고 정지함을 의미하나 좋지 않은 습관임

In [4]:
""" 컨텍스트 관리자를 활용한 DB Dump 사례 """

def stop_database():
    print("systemctl stop postgresql.service")

def start_database():
    print("systemctl start postgresql.service")


class DBHandler:
    def __enter__(self):
        stop_database()
        return self

    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_database()


def db_backup():
    print("pg_dump database")

def main():
    with DBHandler():
        db_backup()

main()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


### 컨텍스트 관리자 구현
+ contextlib 모듈을 활용한 컨텍스트 관리자 구현 가능
    + contextlib.contextmanager 데코레이터를 적용하면 해당 함수의 코드를 컨텍스트 관리자로 변환함
    + contextlib.ContextDecorator 를 사용하면 with문 없이 완전히 독립적인 실행가능
        + 컨텍스트 관리자 내부에서 사용하고자 하는 객체를 얻을 수 없는 단점
        + e.g.) with offline_back() as bp: 처럼 사용할 수 없음
    + with contextlib.suppress(DataConversionException) 은 로직 자체적으로 처리하고 있음을 예외임을 명시하고 실패하지 않도록 함

In [5]:
import contextlib

@contextlib.contextmanager
def db_handler():
    stop_database()
    yield
    start_database()

def main():
    with db_handler():
        db_backup()

main()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


In [6]:
import contextlib

class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        stop_database()

    def __exit__(self, ext_type, ex_value, ex_traceback):
        start_database()

@dbhandler_decorator()
def offline_backup():
    print("pg_dump database")

## 프로퍼티, 속성과 객체 메서드의 다른 타입들

+ 파이썬 객체의 모든 프로퍼티와 함수는 public
+ 따라서 엄격한 강제사항은 없지만 밑줄로 시작하는 속성은 해당 객체에 대해 private을 의미하며 외부에서 호출하지 않기를 기대함

### 파이썬에서의 밑줄
+ 이중 밑줄은 여러 번 확장되는 클래스의 메서드를 이름 충돌 없이 오버라이드하기 위해 만들어짐
+ 이중 밑줄이 아닌 하나의 밑줄을 사용하는 파이썬의 관습을 지킬 것


In [7]:
class Connector:
    def __init__(self, source):
        self.source = source
        self._timeout = 60 # 하나의 밑줄

conn = Connector("postgresql://localhost")

conn._timeout = 70
print(conn.__dict__)

{'source': 'postgresql://localhost', '_timeout': 70}


In [8]:
class Connector:
    def __init__(self, source):
        self.source = source
        self.__timeout = 60 # 두 개의 밑줄

conn = Connector("postgresql://localhost")

conn.__timeout # "접근할 수 없다"가 아닌 "존재하지 않는다"는 오류 발생

AttributeError: 'Connector' object has no attribute '__timeout'

In [10]:
conn._Connector__timeout = 80 # 이름이 바뀌어 있음
print(conn.__dict__)

{'source': 'postgresql://localhost', '_Connector__timeout': 80}


### 프로퍼티

+ 객체의 어떤 속성에 대한 접근을 제어하려는 경우 사용함 (잘못된 정보의 입력 제한 등)
+ 모든 속성에


In [20]:
import re

EMAIL_FORMAT = re.compile(r"[^@]+@[^@]+[^@]+")

def is_valid_email(potentially_valid_email: str):
    return re.match(EMAIL_FORMAT, potentially_valid_email) is not None

class User:
    def __init__(self, username):
        self.username = username
        self._email = None

    @property
    def email(self):
        return self._email

    @email.setter
    def email(self, new_email):
        if not is_valid_email(new_email):
            raise ValueError(f"유효한 이메일이 아니므로 {new_email} 값을 사용할 수 없음")
        self._email = new_email

user = User("shyeon.kang")
user.email = "shyeon.kang@"

ValueError: 유효한 이메일이 아니므로 shyeon.kang@ 값을 사용할 수 없음

In [21]:
user = User("Kang Seonghyeon")
user.email = "Seonghyeon@gmail.com"